In [ ]:
def load_file(filename: str, info_on = False) -> pd.DataFrame:
    """Returns dataframe of complex numbers from a given file

    Parameters
    ----------
    filename : str
        name of the file to be loaded

    Returns
    -------
    DataFrame
        a dataframe of complex numbers
    """
    df = pd.read_csv(filename)
    
    df.a = df.a.astype(complex)
    df.b = df.b.astype(complex)
    df.c = df.c.astype(complex)
    df.d = df.d.astype(complex)
    
    df = df.reset_index()
    
    if(info_on):
        display(df.head())
        display(df.info())
    
    return df

#help(load_file)

In [ ]:
def get_H(a: complex, b: complex, c: complex, d: complex) -> PauliSumOp:
    """Returns a sum of Pauli operators equivalent to the given 2x2 Hamiltonian described by its coefficients

    Parameters
    ----------
    a, b, c, d : complex
        coefficients of the 2x2 Hamiltonian

    Returns
    -------
    PauliSumOp
        a list of Pauli operators with their coefficients
    """
    H = 1/2 * (a + d) * I + 1/2 * (a - d) * Z + 1/2 * (b + c) * X + 1/2 * (b - c) * 1j*Y
    
    return H

#help(get_H)

In [ ]:
def get_U(H: PauliSumOp, t: Union[float, Parameter]) -> EvolvedOp:
    """Returns Unitary operator of time evolution for Hamiltonian H and time t

    Parameters
    ----------
    H : PauliSumOp
        Hamiltonian described by Pauli operators
    t : float or Parameter
        time of the evolution

    Returns
    -------
    EvolvedOp
        unitary matrix corresponding to the evolution of Hamiltonian at time t
    """
    U = (H * t / hbar).exp_i()
    
    return U

#help(get_U)

In [ ]:
def get_ΔE(H: PauliSumOp) -> complex:
    """Returns difference between eigenvalues of a given Hamiltonian

    Parameters
    ----------
    H : PauliSumOp
        Hamiltonian described by Pauli operators

    Returns
    -------
    complex
        difference between eigenvalues of a given Hamiltonian
    """
    H_matrix = H.to_matrix()
    ΔE = np.sqrt((H_matrix[0][0] - H_matrix[1][1])**2 + 4*np.abs(H_matrix[0][1])**2)

    return ΔE

#help(get_ΔE)

In [ ]:
def get_T(H: PauliSumOp) -> float:
    """Returns total time of an evolution for given Hamiltonian

    Parameters
    ----------
    H : PauliSumOp
        Hamiltonian described by Pauli operators

    Returns
    -------
    float
        total time of an evolution for a given Hamiltonian
    """
    ΔE = get_ΔE(H)
    T = h/ΔE

    return float(T)

#help(get_T)

In [ ]:
def get_P(H: PauliSumOp, t: float, initial_state: DictStateFn) -> float:
    """Evolves Hamiltonian and returns probability of observing given state at time t

    Parameters
    ----------
    H : PauliSumOp
        Hamiltonian described by Pauli operators
    t : float
        time for evolution
    initial_state : DictStateFn
        state that we want to measure

    Returns
    -------
    float
        probability of observing given state at time t
    """
    U = get_U(H, t)
    P = np.abs((~initial_state @ U @ initial_state).eval())**2

    return P

#help(get_P)

In [ ]:
def plot_probs(probs0: list, probs1: list, ts: np.ndarray) -> None:
    """Plots given probabilities of measuring states |0> and |1>

    Parameters
    ----------
    probs0 : list
        probabilites of measuring 0
    probs1 : list
        probabilities of measuring 1
    ts : ndarray
        time intervals
    """
    plt.rcParams["figure.figsize"] = (plot_width, plot_height)
    plt.plot(ts, probs0, linewidth=3)
    plt.plot(ts, probs1, linewidth=3)
    plt.xlabel('time')
    plt.ylabel(r'probability of state $|0\rangle$ and state $|1\rangle$')
    plt.title(r'Evolution of states $|0\rangle$ and $|1\rangle$ under $H$')
    plt.legend(["Measurements for '0' ", "Measurements for '1'"])
    plt.grid()
    plt.show()

#help(plot_probs)

---

In [ ]:
def get_times(H: PauliSumOp, periods: float, steps: float) -> np.ndarray:
    """Prints given matrix + total time of evolution & returns points for time intervals of evolution

    Parameters
    ----------
    H : PauliSumOp
        Hamiltonian described by Pauli operators
    periods: float
        number of time periods
    steps: float
        number of steps for time evolution

    Returns
    -------
    ndarray
        an array of points for time intervals
    """
    
    print(H.to_matrix())
    
    T = get_T(H)
    print("Total time of evolution:", T)
    
    ts = np.linspace(0, T * periods, steps)
    return ts

#help(get_times)

In [ ]:
def prepare_qc(H: PauliSumOp, evolution_time: float) -> QuantumCircuit:
    """Prints given matrix + total time of evolution & returns points for time intervals of evolution

    Parameters
    ----------
    H : PauliSumOp
        Hamiltonian described by Pauli operators
    evolution_time : float
        total time of an evolution for the Hamiltonian

    Returns
    -------
    QuantumCircuit
        quantum circuit with Unitary and measurement
    """
    t = Parameter('t')
    
    q = QuantumRegister(1, "q")
    c = ClassicalRegister(1, "c")
    qc = QuantumCircuit(q, c)
    
    if(initial_state == One):
        qc.x(0)
    
    unitary = MatrixEvolution().convert(get_U(H, t))
    unitary = unitary.bind_parameters({ t: evolution_time })

    UMG = unitary.to_circuit().to_gate()
    UMG.label = 'Unitary'

    qc.append(UMG, [0])
    qc.measure(q, c)
    
    return qc

#help(prepare_qc)

In [ ]:
def monitor_jobs(jobs: list) -> None:
    """Monitor state of the jobs

    Parameters
    ----------
    jobs : AerJob
        Hamiltonian described by Pauli operators
    """
    for job in jobs:
        job_monitor(job)
        try:
            if job.error_message() is not None:
                print(job.error_message())
        except:
            pass

#help(monitor_jobs)

---

In [ ]:
def create_dataframe(data: list, mn=True, err=True):
    """Creates DataFrame from given data (probabilities of the measurements)
    
    Parameters
    ----------
    data : list
        data (probabilities) to be transformed into dataframe
    mean : bool
        specifies if mean will be computed and added to the dataframe
    err : bool
        specifies if mean absolute error (mae) will be computed and added to the dataframe
    
    Returns
    -------
    DataFrame
        dataframe with all the given data and optionally also mean and mae values
    """
    runs = len(data)
    labels = ["run_"+ str(i) for i in range(1,runs+1)]
    data = pd.DataFrame(data, index=labels)
    
    if(mn or err):
        mean = pd.Series(data.mean(axis=0), name="mean")
        data = data.append(mean)
    
    if(err):
        error = []
        for i in range(steps):
            error.append(mae(data.iloc[0:-1, i], [data.loc["mean", i]]*runs))
            
        error = pd.Series(error, name="error")
        perc_error = pd.Series(error / mean * 100, name="% error")
        data = data.append(error)
        data = data.append(perc_error)
    
    data.columns =["point_"+ str(i) for i in range(1,steps+1)]
    return data

#help(create_dataframe)

---

In [ ]:
def get_meas_filter(backend: Backend):
    """Returns meas filter (with mitigation matrix) from given data (probabilities of the measurements)
    
    Parameters
    ----------
    backend : Backend
        backend which will be used for running sample circuits
    
    Returns
    -------
    MeasurementFilter
        meas filter to be applied to the circuit to perform mitigation
    """
    
    q = QuantumRegister(1)
    qubit_list = [0]
    meas_calibs, state_labels = complete_meas_cal(qubit_list=qubit_list, qr=q, circlabel='mcal')

    job = execute(meas_calibs, backend=backend, shots=num_shots)
    cal_results = job.result()
    
    meas_fitter = CompleteMeasFitter(cal_results, state_labels, qubit_list=qubit_list, circlabel='mcal')
    meas_filter = meas_fitter.filter
    
    return meas_filter

#help(get_meas_filter)

In [1]:
def print_hamiltonians_info():
    """Prints information about loaded Hamiltonians"""
    for H in HAMILTONIANS:
        print("\n", H)

In [ ]:
def read_res(filename):
    """Returns meas filter (with mitigation matrix) from given data (probabilities of the measurements)
    
    Parameters
    ----------
    filename : str
        backend which will be used for running sample circuits
    
    Returns
    -------
    MeasurementFilter
        meas filter to be applied to the circuit to perform mitigation
    """
    return pd.read_csv("../OUTPUT/DATAFRAMES/"+ filename +".csv", index_col="Unnamed: 0")

In [ ]:
# print("... running of {functions.ipynb}  successfully finished \U0001F389")